# A Guided Tour of Ray Core: Remote Classes

[*Remote Classes*](https://docs.ray.io/en/latest/walkthrough.html#remote-classes-actors)
involve using a `@ray.remote` decorator on a class. 

This implements an [*actor*](https://patterns.eecs.berkeley.edu/?page_id=258) pattern, with properties: *stateful*, *message-passing semantics*

---

First, let's start Ray…

In [ ]:
import logging
import ray

ray.init(
    ignore_reinit_error=True,
    logging_level=logging.ERROR,
)

## Remote Classes example

To start, we'll define a class and use the decorator:

In [ ]:
@ray.remote
class Counter:
    def __init__ (self):
        self.value = 0

    def increment (self):
        self.value += 1
        return self.value

Now use this class `Counter` to create an actor:

In [ ]:
%%time

counter = Counter.remote()

Then call the actor:

In [ ]:
%%time

obj_ref = counter.increment.remote()
ray.get(obj_ref)

Use list comprehension to show the state being maintained in the actor

In [ ]:
%%time

f_list = [counter.increment.remote() for _ in range(3)]

In [ ]:
%%time 

print(ray.get(f_list))


Let's use another Actor class and create multiple instances associated with a distinct attribute, such as a name.

In [ ]:
from random import randint

@ray.remote
class GoalsScored:
    def __init__ (self, player) -> None:
        self._goals = 0
        self._player = player

    def score (self) -> object:
        self._goals += randint(1, 5)
        return self._goals
       
    def player(self) -> str:
        return self._player

Define three Actors: Rolando, Neymar, Messi

In [ ]:
%%time 

ronaldo = GoalsScored.remote("Ronaldo")
neymar = GoalsScored.remote("Neymar")
messi = GoalsScored.remote("Messi")

In [ ]:
%%time

ronaldo_ref = ronaldo.score.remote()
neymar_ref = neymar.score.remote()
messi_ref  = messi.score.remote()


Again, use list comprehension to iterate over each Actor instances, along with object_ref
for their scores, maintained by each distincgive actor.

In [ ]:
for ref, ref_obj in [(ronaldo, ronaldo_ref), (neymar, neymar_ref), (messi, messi_ref) ]:
   print(f"Player: {ray.get(ref.player.remote())} and goals scored: {ray.get(ref_obj)}")

Finally, shutdown Ray

In [ ]:
ray.shutdown()

---
## References

["A Universal Modular Actor Formalism for Artificial Intelligence"](https://www.ijcai.org/Proceedings/73/Papers/027B.pdf)  
Carl Hewitt, Peter Bishop, Richard Steiger  
*IJCAI* (1973)